# Demo: Data Uploader for UFS Datasets to Cloud Data Storage

### __Purpose:__ 

The purpose of this program is to transfer the Unified Forecast Sytstem (UFS) input and baseline datasets residing within the RDHPCS to cloud data storage via chaining API calls to communicate with cloud data storage buckets. The program will support the data required for the current UFS Weather Model (UFS WM) deployed within the RDHPCS as well as support the NOAA development team's data management in maintaining only the datasets committed within the latest N months of their UFS development code (once the program is integrated into Jenkins).

According to Amazon AWS, the following conditions need to be considered when transferring data to cloud data storage:
- Largest object that can be uploaded in a single PUT is 5 GB.
- Individual Amazon S3 objects can range in size from a minimum of 0 bytes to a maximum of 5 TB.
- For objects larger than 100 MB, Amazon recommends using the Multipart Upload capability.
- The total volume of data in a cloud data storage bucket are unlimited.

Tools which could be be utilized to perform data transferring & partitioning (Multipart Upload/Download) are: 
- AWS SDK
- AWS CLI
- AWS S3 REST API

All of the AWS provided tools are built on Boto3. 

In this demontration, the framework will implement Python AWS SDK for transferring the UFS datasets from the RDHPCS, Orion, to the cloud data storage with low latency. 

The AWS SDK will be implemented for the following reasons:
- To integrate with other python scripts.
- AWS SDK carries addition capabilities/features for data manipulation & transferring compare to the aforementioned alternate tools.

### __Capabilities:__ 

The framework will be able to perform the following actions:

- Multi-threading & partitioning to the datasets to assist in the optimization in uploading performance of the datasets from on-prem to cloud. 

### __Future Capabilities:__  

The program can be used as a skeletal framework for transferring future datasets of interest (e.g. SRW data, MRW data, etc). In addition, it can be integrated with the UFS tracker bot (https://github.com/NOAA-EPIC/ufs-dev_data_timestamps) & Jenkins to automate the data transferring process as new datasets are being committed & pushed to the UFS-WM repository develop branch.


### __Sample Datasets to Transfer:__
There are two scenarios that will need to be considered when storing the UFS data in cloud:

- Datasets to be stored in cloud need to support NOAA's development team. Datasets residing within the Cloud as well as RDHPCS must support their development team's latest 2 months of developing code. 


| UFS MODEL DEVELOPMENT VERSIONS| BASELINE DATA | INPUT DATA | WW3 INPUT DATA | BM_IC |
| :- | :- | :- | -: | :-: |
| Supports NOAA Dev Team Versons (since 03-04-22)| 20220304 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207 
| Supports NOAA Dev Team Versons (since 03-16-22)| 20220316 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207 |
| Supports NOAA Dev Team Versons (since 03-18-22)| 20220318 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207 |
| Supports NOAA Dev Team Versions (since 03-18-22)| 20220321 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207|
| Supports NOAA Dev Team Versions (since 03-18-29)| 20220329 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207|

- Datasets to be stored need to support the UFS weather model develop branch code revision, which was pulled last year October 2021 by the EPIC's Platform team. These datasets are:

| UFS MODEL DEVELOPMENT VERSIONS| BASELINE DATA | INPUT DATA | WW3 INPUT DATA | BM_IC |
| :- | :- | :- | -: | :-: |
| Supports UFS Model Version Deployed in CSPs| 20220207 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20210717 |

<img src="./images/DataVersionsZach&JongAreUsing.png">

### __Environment Setup:__

1. Install miniconda on your machine. Note: Miniconda is a smaller version of Anaconda that only includes conda along with a small set of necessary and useful packages. With Miniconda, you can install only what you need, without all the extra packages that Anaconda comes packaged with:

Download latest Miniconda (e.g. 3.9 version):
- __wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh__

Check integrity downloaded file with SHA-256:
- __sha256sum Miniconda3-py39_4.9.2-Linux-x86_64.sh__

Reference SHA256 hash in following link: https://docs.conda.io/en/latest/miniconda.html

Install Miniconda in Linux:
- __bash Miniconda3-py39_4.9.2-Linux-x86_64.sh__

Next, Miniconda installer will prompt where do you want to install Miniconda. Press ENTER to accept the default install location i.e. your $HOME directory. If you don't want to install in the default location, press CTRL+C to cancel the installation or mention an alternate installation directory. If you've chosen the default location, the installer will display “PREFIX=/var/home/<user>/miniconda3” and continue the installation.

For installation to take into effect, run the following command: 
- __source ~/.bashrc__

Next, you will see the prefix (base) in front of your terminal/shell prompt. Indicating the conda's base environment is activated.

2.	Once you have conda installed on your machine, perform the following to create a conda environment:

To create a new environment (if a YAML file is not provided)
- __conda create -n [Name of your conda environment you wish to create]__

__(OR)__

To ensure you are running Python 3.9:
- __conda create -n myenv Python=3.9__

__(OR)__

To create a new environment from an existing YAML file (if a YAML file is provided):
- __conda env create -f environment.yml__

__*Note:__ A .yml file is a text file that contains a list of dependencies, which channels a list for installing dependencies for the given conda environment. For the code to utilize the dependencies, you will need to be in the directory where the environment.yml file lives.

4.	Activate the new environment via: 
- __conda activate [Name of your conda environment you wish to activate]__

5.	Verify that the new environment was installed correctly via:
- __conda info --env__

__*Note:__
- From this point on, must activate conda environment prior to .py script(s) or jupyter notebooks execution
using the following command: __conda activate__
- To deactivate a conda environment: 
    - __conda deactivate__

#### ___Link Home Directory to Dataset Location on RDHPCS Platform___ 

6.	Unfortunately, there is no way to navigate to the /work/ filesystem from within the Jupyter interface. The best way to workaround is to create a symbolic link in your home folder that will take you to the /work/ filesystem. Run the following command from a linux terminal on Orion to create the link: 

    - __ln -s /work /home/[Your user account name]/work__

Now, when you navigate to the __/home/[Your user account name]/work__ directory in Jupyter, it will take you to the __/work__ folder. Allowing you to obtain any data residing within the __/work__ filesystem that you have permission to access from Jupyter. This same procedure will work for any filesystem available from the root directory. 

__*Note:__ On Orion, user must sym link from their home directory to the main directory containing the datasets of interest.

#### ___Open & Run Data Analytics Tool on Jupyter Notebook___

7.	Open OnDemand has a built-in file explorer and file transfer application available directly from its dashboard via ...
    - Login to https://orion-ood.hpc.msstate.edu/ 
    - In the Open OnDemand Interface, select __Interactive Apps__ > __Jupyter Notbook__
    - Set the following configurations to run Jupyter:


#### ___Additonal Information___

__To create a .yml file, execute the following commands:__

- Activate the environment to export: 
    - __conda activate myenv__

- Export your active environment to a new file:
    - __conda env export > [ENVIRONMENT FILENAME].yml__


### __Reference(s)__
Latest UFS Weather Model Guide:
- https://ufs-weather-model.readthedocs.io/en/latest/InputsOutputs.html


# Demo 1: Data Locality Extractor from Source

#### Test Sample
The script will read from the data tracker bot's output pickle file. The test sample generated by the data tracker bot will reside in **./data_from_ts_tracker/latest_rt.sh.pk** at this time until Jenkins is connected to the RDHPCS.

In this demonstration, datasets were transferred to the cloud data storage to support the following developing UFS-WMs of interest.

| UFS MODEL DEVELOPMENT VERSIONS| BASELINE DATA | INPUT DATA | WW3 INPUT DATA | BM_IC |
| :- | :- | :- | -: | :-: |
| Supports UFS Model Version Deployed in CSPs| 20220207 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20210717 |
| Supports NOAA Dev Team Versons (since 03-04-22)| 20220304 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207 
| Supports NOAA Dev Team Versons (since 03-16-22)| 20220316 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207 |
| Supports NOAA Dev Team Versons (since 03-18-22)| 20220318 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207 |
| Supports NOAA Dev Team Versions (since 03-18-22)| 20220321 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207|
| Supports NOAA Dev Team Versions (since 03-18-29)| 20220329 | input-data-20211210  | WW3_input_data_20211113 | BM_IC-20220207|


### Obtain directories for the datasets tracked by the data tracker bot. 

**TODO: FOR YEAR 2, will capture the date at which the ts were extracted and then write a script which will add a sliding filter window to capture only the latest 2 months of development code.

In [ ]:
# if __name__ == '__main__': 
    
#     # Module for extracting data from source.
#     from get_timestamp_data import GetTimestampData
    
#     # Establish locality of where the dataseta are sourced.
#     linked_home_dir = "/home/schin/work"
#     orion_rt_data_dir = linked_home_dir + "/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/"

#     # Filter to tracker log's timestamps & extract their corresponding UFS input & baseline file directories.
#     filter2tracker_ts_datasets = GetTimestampData(orion_rt_data_dir, None).filter2tracker_ts_datasets
# filter2tracker_ts_datasets.keys()

### Obtain directories for the datasets requested by the user.

In [1]:
if __name__ == '__main__': 
    
    # Module for extracting data from source.
    from get_timestamp_data import GetTimestampData
    
    # Establish locality of where the dataseta are sourced.
    linked_home_dir = "/home/schin/work"
    orion_rt_data_dir = linked_home_dir + "/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/"
    
    # Select timestamp dataset to transfer from RDHPCS on-disk to cloud
    input_ts, bl_ts, ww3_input_ts, bmic_ts = [], ['develop-20220502'], [], []
    filter2specific_ts_datasets = GetTimestampData(orion_rt_data_dir, None).get_specific_ts_files(input_ts, bl_ts, ww3_input_ts, bmic_ts)
filter2specific_ts_datasets.keys()

All Primary Dataset Folders & Files In Main Directory (/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/):

['develop-20220321', 'develop-20220329', 'develop-20220318', 'ufs-public-release-v2-20210212', 'develop-20220406', 'ufs-public-release-v2-20210208', 'develop-20220322', 'BM_IC-20220207', 'develop-20220316', 'develop-20220401', 'develop-20220325', 'develop-20220414', 'develop-20220408', 'develop-20220502', 'develop-20220425', 'develop-20220420', 'develop-20220421', 'develop-20220328', 'input-data-20220414', 'develop-20220304', 'BM_IC-20210717', 'develop-20220418']


dict_keys(['BL_DATE'])

In [2]:
# Selected timestamp dataset to transfer from RDHPCS on-disk to cloud
filter2specific_ts_datasets

defaultdict(list,
            {'BL_DATE': ['develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/sfc.nest02.f006.nc',
              'develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/sfcf006.nc',
              'develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/atm.nest02.f006.nc',
              'develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/atmf006.nc',
              'develop-20220502/INTEL/hafs_regional_atm_wav/sfcf006.nc',
              'develop-20220502/INTEL/hafs_regional_atm_wav/out_pnt.ww3',
              'develop-20220502/INTEL/hafs_regional_atm_wav/out_grd.ww3',
              'develop-20220502/INTEL/hafs_regional_atm_wav/atmf006.nc',
              'develop-20220502/INTEL/cpld_control_c192_p8/sfcf036.tile6.nc',
              'develop-20220502/INTEL/cpld_control_c192_p8/atmf036.tile2.nc',
              'develop-20220502/INTEL/cpld_control_c192_p8/atmf036.tile4.nc',
              'develop-20220502/INTEL/cpld_control_c192_p8/

# Demo 2: Multipart Upload of Extracted Data to Cloud

### Upload datasets tracked by the data tracker bot.

In [ ]:
# if __name__ == '__main__': 
#     from progress_bar import ProgressPercentage
#     from upload_data import UploadData
#     linked_home_dir = "/home/schin/work"
#     orion_rt_data_dir = linked_home_dir + "/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/"
#     uploader_wrapper = UploadData(orion_rt_data_dir, filter2tracker_ts_datasets, use_bucket='rt')
#     uploader_wrapper.upload_files2cloud()

### Upload datasets by timestamps as requested by the user.
- In this scenario, used when transferring data files required for the UFS-WM currently deployed in the CSPs.

In [3]:
# if __name__ == '__main__': 
#     from progress_bar import ProgressPercentage
#     from upload_data import UploadData
#     linked_home_dir = "/home/schin/work"
#     orion_rt_data_dir = linked_home_dir + "/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/"
#     uploader_wrapper = UploadData(orion_rt_data_dir, filter2specific_ts_datasets, use_bucket='rt')
#     uploader_wrapper.upload_files2cloud()


/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/sfc.nest02.f006.nc  58199637 / 58199637.0  (100.00%)Processing Time (min): 0.026953613758087157

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/sfcf006.nc  30314997 / 30314997.0  (100.00%)Processing Time (min): 0.010575759410858154

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/atm.nest02.f006.nc  312080534 / 312080534.0  (100.00%)Processing Time (min): 0.06611098051071167

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_regional_specified_moving_1nest_atm/atmf006.nc  179630172 / 179630172.0  (100.00%)Processing Time (min): 0.03943541447321574

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_regional_atm_wav/sfcf006.nc  161687208 / 161687208.0  (100.00%)Processin

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c192_p8/RESTART/phy_data.tile1.nc  154851708 / 154851708.0  (100.00%)Processing Time (min): 0.04035422404607137

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c192_p8/RESTART/iced.2021-03-23-64800.nc  527526388 / 527526388.0  (100.00%)Processing Time (min): 0.09034786224365235

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c192_p8/RESTART/fv_tracer.res.tile5.nc  1310907936 / 1310907936.0  (100.00%)Processing Time (min): 0.21632789770762126

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c192_p8/RESTART/coupler.res  300 / 300.0  (100.00%)Processing Time (min): 0.0031681021054585775

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c192_p8/RESTART/fv_srf_wnd.res.tile5.nc  609832 / 609832.0  (100.00%)Processing Time (min): 0.003833

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_p8_lndp/GFSFLX.GrbF21  6078541 / 6078541.0  (100.00%)Processing Time (min): 0.005649248758951823

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_p8_lndp/GFSFLX.GrbF48  6239832 / 6239832.0  (100.00%)Processing Time (min): 0.005986146132151286

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_p8_lndp/GFSFLX.GrbF00  2794062 / 2794062.0  (100.00%)Processing Time (min): 0.004145769278208414

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_p8_lndp/sfcf048.nc  46339890 / 46339890.0  (100.00%)Processing Time (min): 0.020182470480600994

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_p8_lndp/atmf021.nc  483652406 / 483652406.0  (100.00%)Processing Time (min): 0.09218382438023885

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_p

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_iovr4/atmf000.nc  483652394 / 483652394.0  (100.00%)Processing Time (min): 0.08821422259012858

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_iovr4/sfcf000.nc  44570321 / 44570321.0  (100.00%)Processing Time (min): 0.015979031721750896

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_iovr4/GFSPRS.GrbF00  66825795 / 66825795.0  (100.00%)Processing Time (min): 0.022066394488016765

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_iovr4/atmf024.nc  483652394 / 483652394.0  (100.00%)Processing Time (min): 0.09061807791392008

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_iovr4/GFSPRS.GrbF21  65674287 / 65674287.0  (100.00%)Processing Time (min): 0.02088513771692912

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_iovr4/GF

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c384_p8/RESTART/phy_data.tile1.nc  614196604 / 619341180.0  (99.17%)

...

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_c192/atmf024.nc  1954754874 / 1954754874.0  (100.00%)Processing Time (min): 0.34908161163330076

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_c192/GFSFLX.GrbF24  24082644 / 24082644.0  (100.00%)Processing Time (min): 0.022636655966440836

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_c192/sfcf024.nc  178635473 / 178635473.0  (100.00%)Processing Time (min): 0.06413125991821289

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_c192/GFSPRS.GrbF24  235118793 / 235118793.0  (100.00%)Processing Time (min): 0.050430476665496826

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_thompson_no_aero/GFSFLX.GrbF00  2704561 / 2704561.0  (100.00%)Processing Time (min): 0.005770961443583171

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/con

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/rrfs_conus13km_radar_tten_warm/sfcf000.nc  291350394 / 291350394.0  (100.00%)Processing Time (min): 0.055181574821472165

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/rrfs_conus13km_radar_tten_warm/atmf001.nc  304287476 / 304287476.0  (100.00%)Processing Time (min): 0.05783303578694662

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/rrfs_conus13km_radar_tten_warm/atmf002.nc  304287476 / 304287476.0  (100.00%)Processing Time (min): 0.05786478519439697

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hrrr_control/GFSFLX.GrbF21  6033054 / 6033054.0  (100.00%)Processing Time (min): 0.0073764999707539875

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hrrr_control/GFSFLX.GrbF00  2601638 / 2601638.0  (100.00%)Processing Time (min): 0.004659624894460043

/home/schin/work/noaa/nems/emc.nemspara/RT/N

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hrrr_control/RESTART/sfc_data.tile1.nc  10053216 / 10053216.0  (100.00%)Processing Time (min): 0.005759990215301514

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hrrr_control/RESTART/fv_tracer.res.tile3.nc  60883048 / 60883048.0  (100.00%)Processing Time (min): 0.02742632230122884

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hrrr_control/RESTART/fv_core.res.tile5.nc  37610116 / 37610116.0  (100.00%)Processing Time (min): 0.016765435536702473

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hrrr_control/RESTART/sfc_data.tile6.nc  10053216 / 10053216.0  (100.00%)Processing Time (min): 0.007099286715189616

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/control_csawmg/GFSFLX.GrbF00  2640294 / 2640294.0  (100.00%)Processing Time (min): 0.004447448253631592

/home/schin/work/noaa/nems/emc.nems

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_global_multiple_4nests_atm/atm.nest05.f006.nc  210108443 / 210108443.0  (100.00%)Processing Time (min): 0.07577509085337321

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_global_multiple_4nests_atm/atmf006.nc  129627865 / 129627865.0  (100.00%)Processing Time (min): 0.06480560302734376

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_global_multiple_4nests_atm/sfc.nest05.f006.nc  62917795 / 62917795.0  (100.00%)Processing Time (min): 0.19451640049616495

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_global_multiple_4nests_atm/atm.nest03.f006.nc  173951439 / 173951439.0  (100.00%)Processing Time (min): 0.04542712370554606

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/hafs_global_multiple_4nests_atm/sfc.nest04.f006.nc  65458887 / 65458887.0  (100.00%)Processing Time (min)

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c96_p8/RESTART/coupler.res  300 / 300.0  (100.00%)Processing Time (min): 0.0029032190640767416

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c96_p8/RESTART/fv_srf_wnd.res.tile5.nc  165928 / 165928.0  (100.00%)Processing Time (min): 0.0029114127159118653

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c96_p8/RESTART/MOM.res.nc  509316754 / 509316754.0  (100.00%)Processing Time (min): 0.12540442148844402

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c96_p8/RESTART/fv_srf_wnd.res.tile4.nc  165928 / 165928.0  (100.00%)Processing Time (min): 0.0036467512448628745

/home/schin/work/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/develop-20220502/INTEL/cpld_control_c96_p8/RESTART/sfc_data.tile3.nc  9834052 / 9834052.0  (100.00%)Processing Time (min): 0.008315463860829672

/home/schin/

# Consolidated Demo: Extract Data Localities & Upload to Cloud

### Extract & upload datasets (input data + baseline data + IC data) as requested by the user.

In [ ]:
# Obtain directories & upload to cloud for the datasets requested by the user.
from transfer_specific_data import TransferSpecificData
input_ts, bl_ts, ww3_input_ts, bmic_ts = [], ['develop-20220406'], [], []
TransferSpecificData(input_ts, bl_ts, ww3_input_ts, bmic_ts, linked_home_dir="/home/schin/work", platform="orion")

### Extract & upload datasets tracked by the data tracker bot.

In [ ]:
# # Obtain directories & upload to cloud for the datasets tracked by the data tracker bot.
# from transfer_bot_data import TransferBotData
# TransferBotData(linked_home_dir="/home/schin/work", platform="orion")

# Demo: Upload a Single Data File of Interest

In [ ]:
# if __name__ == '__main__': 
#     from progress_bar import ProgressPercentage
#     from upload_data import UploadData
#     linked_home_dir = "/home/schin/work"
#     orion_rt_data_dir = linked_home_dir + "/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/"
#     uploader_wrapper = UploadData(orion_rt_data_dir, file_relative_dirs=None, use_bucket='rt')
    
#     # Upload a Single Data File of Interest
#     file_dir = 'input-data-20211210/fv3_regional_c768/INPUT/grid.tile7.halo4.nc'
#     uploader_wrapper.upload_single_file(file_dir)

# Demo: Delete a File

In [ ]:
# if __name__ == '__main__': 
#     from progress_bar import ProgressPercentage
#     from upload_data import UploadData
#     linked_home_dir = "/home/schin/work"
#     orion_rt_data_dir = linked_home_dir + "/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/"
#     uploader_wrapper = UploadData(orion_rt_data_dir, file_relative_dirs=None, use_bucket='rt')
#     file_dir = 'input-data-20211210/fv3_regional_c768/INPUT/grid.tile7.halo4.nc'
#     key_path = file_dir
#     uploader_wrapper.purge(key_path)

# Demo: Delete Objects with Key Prefix

In [ ]:
# if __name__ == '__main__': 
#     from progress_bar import ProgressPercentage
#     from upload_data import UploadData
#     linked_home_dir = "/home/schin/work"
#     orion_rt_data_dir = linked_home_dir + "/noaa/nems/emc.nemspara/RT/NEMSfv3gfs/"
#     uploader_wrapper = UploadData(orion_rt_data_dir, file_relative_dirs=None, use_bucket='rt')
#     key_prefix = 'develop-20220418'
#     uploader_wrapper.purge_by_keyprefix(key_prefix)


# Demo: Copy Objects & Delete with Key Prefix
AWS CLI copies the objects to the target folder and then removes the original file. There is no “move” action in S3.

In [ ]:
# if __name__ == '__main__': 
#     from progress_bar import ProgressPercentage
#     from upload_data import UploadData
#     uploader_wrapper = UploadData(orion_rt_data_dir, file_relative_dirs=None, use_bucket='rt')
#     source_key_path = '####' 
#     new_key_path = '###'
#     uploader_wrapper.rename_s3_keys(source_key_path, new_key_path)

# Demo: Get List of All Keys in UFS RT S3 Bucket

In [ ]:
# if __name__ == '__main__': 
#     from progress_bar import ProgressPercentage
#     from upload_data import UploadData
#     uploader_wrapper = UploadData(orion_rt_data_dir, file_relative_dirs=None, use_bucket='rt')
#     all_bucket_objects = uploader_wrapper.get_all_s3_keys()
# all_bucket_objects

In [ ]:
# Write list to text file and save to directory
# with open('filename].[file_format]', 'w') as f:
#     for item in all_bucket_objects:
#         f.write("%s\n" % item)